In [ ]:
### THIS CELL SETS UP THE GOOGLE COLAB ENVIRONMENT. 
### IF RUNNING THIS NOTEBOOK LOCALLY, IT MAY BE SAFELY DELETED.

#@title Install software

#@markdown Install the software by pressing the _Play_ button on the left.

try:
    import google.colab
    RUNNING_IN_COLAB = True
except ImportError:
    RUNNING_IN_COLAB = False
except Exception as e: 
    err = "Could not figure out if runnning in a colab notebook\n"
    raise Exception(err) from e

if RUNNING_IN_COLAB:
    !pip install dataprob


In [ ]:
%matplotlib inline
import dataprob
import numpy as np
from scipy import stats

def multi_gaussian(params,num_gaussians,x):
    """
    Generate a multi-guassian.

    Parameters
    ----------
    params : numpy.ndarray
        float numpy array that is num_gaussians*3 long. this encodes the
        gaussian [mean1,std1,area1,mean2,std2,area2,...meanN,stdN,areaN]
        shape parameters
    num_gaussians : int
        number of gaussians in the params array
    x : numpy.ndarray
        calculate guassians over the values in x 

    Returns
    -------
    out : numpy.ndarray
        sum of the pdfs for the gaussians in params calculated over x
    """

    # Create output array
    out = np.zeros(len(x),dtype=float)

    # For each gaussian
    for i in range(num_gaussians):

        # Grab the shape parameters
        mean = params[i*3]
        std = params[i*3 + 1]
        area = params[i*3 + 2]

        # Add this to out
        out += area*stats.norm(loc=mean,scale=std).pdf(x)

    return out
    
gen_params = {"params":np.array([5,0.3,10,6,1.5,10]),
              "num_gaussians":2}

err = 0.25
num_points = 50

x = np.linspace(0,10,num_points)
y_obs = multi_gaussian(x=x,**gen_params) + np.random.normal(0,err,num_points)
y_std = err

test_fcn = multi_gaussian
non_fit_kwargs = {"x":x,
                  "num_gaussians":2}

f = dataprob.setup(some_function=test_fcn,
                   method="ml",
                   fit_parameters=["m0","s0","a0","m1","s1","a1"],
                   non_fit_kwargs=non_fit_kwargs,
                   vector_first_arg=True)

f.param_df.loc[["m0","s0","a0","m1","s1","a1"],"guess"] = [5,1,1,7,1,1]
f.param_df.loc["s0","lower_bound"] = 0
f.param_df.loc["s1","lower_bound"] = 0
f.param_df.loc["a0","lower_bound"] = 0
f.param_df.loc["a1","lower_bound"] = 0

f.fit(y_obs=y_obs,
      y_std=y_std)


fig = dataprob.plot_summary(f)



In [ ]:
fig = dataprob.plot_corner(f,filter_params=["a"])


In [ ]:
f.fit_df

In [ ]:
f.fit_quality

### Compare input and fit parameters

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(1,figsize=(6,6))

x = np.array(list(gen_params["params"]))
y = np.array(f.fit_df.loc[:,"estimate"])
yerr = np.array(f.fit_df.loc[:,"std"])

combined = list(x)
combined.extend(y)
span = (np.max(combined) - np.min(combined))
extend_by = 0.1*span
offset = 0.03*span
min_value = np.min(combined) - extend_by
max_value = np.max(combined) + extend_by

ax.scatter(x,y,s=20,edgecolor='none',facecolor='red',zorder=1)
ax.errorbar(x=x,y=y,yerr=yerr,lw=0,elinewidth=1,capsize=3,color='black',zorder=2)

for i, k in enumerate(f.fit_df["name"]):
    ax.text(x=x[i] - offset,y=y[i] + offset,s=k)


ax.plot((min_value,max_value),(min_value,max_value),'--',color='gray',zorder=0)

ax.set_xlim(min_value,max_value)
ax.set_ylim(min_value,max_value)
ax.set_xlabel("input parameter value")
ax.set_ylabel("estimated parameter value")